In [2]:
from IPython.core.display import HTML
from IPython.display import Image, display
display(HTML("<style>.container { width:100% !important; }</style>"))
import config
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
from imageio import imwrite
import yaml
from fastai.core import Path
from fastai.vision import load_learner, defaults
import torch as torch
%matplotlib inline
%reload_ext autoreload
%autoreload 2
defaults.device = torch.device('cpu')


from tagging_tool_utils import *
from image_graph import *
import ipywidgets as widgets
letter_mappings = yaml.load(open('nbs/letter_maps.yaml'))

In [3]:
pic_name = 'GK_RDR_PG3_2'

In [4]:
gr_path = Path('greek_pages/page_graphs/')
im_path = Path('greek_pages/page_images/')
gname = pic_name + '.txt'
im_name = pic_name + '.jpeg'

model_name = 'rn_34.pkl'
config.model = load_learner('models/', model_name)
letter_path = Path('lgi_data')
config.letter_dest = letter_path/'new_data'
get_letter_dict()

config.img_arr = get_image_array(im_path/im_name)

In [5]:
yaml_pth = Path('lgi_data/')
with open(yaml_pth/'new_components.yaml', 'rb') as f:
    new_components = yaml.load(f)
config.components = new_components
config.itr = 0
config.temp_path = yaml_pth
config.rows, config.cols = config.img_arr.shape
config.letter_mappings = letter_mappings

In [6]:
lb, ub, lbr, ubr = get_bounds(config.itr)

In [7]:
imwrite(config.temp_path/'temp_letter.jpg', config.img_arr[lbr:ubr,lb:ub])
file = open(config.temp_path/"temp_letter.jpg", "rb")
image = file.read()
config.image = widgets.Image(
    value=image,
    format='png',
    width=300,
    height=400,
)

In [8]:
letter_button = widgets.Button(description='Update Letter')
skip_button = widgets.Button(description='Skip')
context_button = widgets.Button(description='Get Context')
reset_context_button = widgets.Button(description='Reset Context')
config.output = widgets.Output()
config.letter_options = widgets.Dropdown(options=[''])
config.letter_options.options =  [''] + get_top_preds(config.temp_path/"temp_letter.jpg")
config.textbox = widgets.Text()
    
letter_button.on_click(update_letter)
skip_button.on_click(skip)
context_button.on_click(get_context)
reset_context_button.on_click(reset_context)
# display
widgets.VBox([config.image,config.textbox,config.letter_options,letter_button,skip_button,context_button,reset_context_button,config.output])

IndexError: list index out of range

In [8]:
yaml.dump(letter_mappings,open('nbs/letter_maps.yaml','w'))